In [1]:
from sklearn.preprocessing import StandardScaler
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np


# Đọc dữ liệu

In [2]:
df = pd.read_csv('dataset_cleaned.csv')

df.head()


,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,0,0.676,0.4610,...,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.715,87.917,4,acoustic
1,1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,0,0.420,0.1660,...,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.267,77.489,4,acoustic
2,2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,0,0.438,0.3590,...,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.120,76.332,4,acoustic
3,3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,0,0.266,0.0596,...,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.143,181.740,3,acoustic
4,4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,0,0.618,0.4430,...,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.167,119.949,4,acoustic


# họn các đặc trưng âm thanh
features = ['danceability', 'energy', 'loudness', 'speechiness', 
             'acousticness', 'instrumentalness', 'liveness', 
             'valence', 'tempo']

- Giải thích:

Danh sách này chứa 9 đặc trưng liên quan đến âm thanh của bài hát.

Các giá trị này thể hiện tính “tương đồng” về cảm giác nghe — ví dụ:

danceability: độ dễ nhảy

energy: năng lượng, độ mạnh

loudness: độ to

valence: cảm xúc tích cực

tempo: nhịp độ (BPM)

- Mục đích:

Lấy các đặc trưng liên quan trực tiếp đến âm thanh để mô hình học ra không gian “ẩn” (latent space) biểu diễn các bài hát tương tự nhau.

In [3]:
features = ['danceability','energy','loudness','speechiness','acousticness',
            'instrumentalness','liveness','valence','tempo']

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(df[features])

# Giới thiệu mô hình Autoencoder

## 1️ Khái niệm
**Autoencoder** là một mô hình **mạng nơ-ron nhân tạo (Artificial Neural Network)** được thiết kế nhằm **học biểu diễn nén (compressed representation)** của dữ liệu đầu vào.  
Nói cách khác, Autoencoder cố gắng **mã hóa (encode)** dữ liệu thành một dạng biểu diễn có số chiều nhỏ hơn, sau đó **giải mã (decode)** để tái tạo lại dữ liệu ban đầu.

>  Mục tiêu của mô hình là tái tạo dữ liệu đầu vào sao cho càng giống dữ liệu gốc càng tốt, thông qua việc học các đặc trưng tiềm ẩn quan trọng nhất.

---

## 2️ Cấu trúc của Autoencoder

Autoencoder gồm **3 phần chính**:

| Thành phần | Vai trò | Giải thích |
|-------------|----------|------------|
|  **Encoder (Bộ mã hóa)** | Nén dữ liệu | Biến đổi dữ liệu đầu vào có chiều cao thành một vector có chiều thấp hơn — gọi là **biểu diễn ẩn (latent representation)**. |
|  **Latent Space (Không gian ẩn)** | Biểu diễn đặc trưng | Lưu trữ các đặc trưng quan trọng nhất mà mô hình học được từ dữ liệu gốc. |
|  **Decoder (Bộ giải mã)** | Tái tạo dữ liệu | Dùng thông tin trong không gian ẩn để tái tạo lại dữ liệu đầu vào. |


---

## 3️ Cách hoạt động
1. **Encoder** học cách biểu diễn dữ liệu đầu vào trong không gian ẩn có kích thước nhỏ hơn.  
2. **Decoder** học cách khôi phục lại dữ liệu ban đầu từ không gian ẩn.  
3. Quá trình huấn luyện được tối ưu bằng cách **giảm lỗi tái tạo** giữa đầu vào và đầu ra.  

---

## 4️ Ứng dụng trong hệ thống gợi ý
Trong bài toán **gợi ý bài hát**, Autoencoder giúp:
- **Tự động trích xuất đặc trưng âm nhạc tiềm ẩn** (latent features) từ các đặc trưng như `energy`, `danceability`, `tempo`, `valence`,...  
- Các đặc trưng này phản ánh **phong cách hoặc “chất” của bài hát**, giúp mô hình hiểu mối quan hệ giữa các bài hát tương tự.  
- Sau khi huấn luyện, ta dùng **Encoder** để lấy vector đặc trưng của từng bài hát và tính **độ tương đồng cosine** giữa chúng để **gợi ý bài hát tương tự**.

---

## 5️ Ưu điểm của Autoencoder
Không cần gán nhãn dữ liệu (Unsupervised Learning)  
**Giảm nhiễu và trích chọn đặc trưng tự động**  
Học được **quan hệ phi tuyến phức tạp** mà PCA hay K-Means không thể nắm bắt  
Cải thiện **độ chính xác** trong gợi ý bài hát, phim, hoặc sản phẩm tương tự  

---



In [ ]:
input_dim = X_scaled.shape[1]
# Xây dựng mô hình Autoencoder
autoencoder = models.Sequential([
    layers.Input(shape=(input_dim,)),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),  # lớp ẩn nén đặc trưng
    layers.Dense(64, activation='relu'),
    layers.Dense(input_dim, activation='sigmoid')
])
# Biên dịch mô hình
autoencoder.compile(optimizer='adam', loss='mse')
autoencoder.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         2,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 9)              │           585 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,417 (21.16 KB)

 Trainable params: 5,417 (21.16 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Huấn luyện mô hình
autoencoder.fit(X_scaled, X_scaled, epochs=50, batch_size=64, validation_split=0.1, verbose=1)


Epoch 1/50
1604/1604 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - loss: 0.0041 - val_loss: 6.4525e-04
Epoch 2/50
1604/1604 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 4.0211e-04 - val_loss: 2.8403e-04
Epoch 3/50
1604/1604 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 2.2323e-04 - val_loss: 2.9668e-04
Epoch 4/50
1604/1604 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 1.7323e-04 - val_loss: 1.7910e-04
Epoch 5/50
1604/1604 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 1.4186e-04 - val_loss: 1.1645e-04
Epoch 6/50
1604/1604 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 1.2700e-04 - val_loss: 1.1515e-04
Epoch 7/50
1604/1604 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 1.1718e-04 - val_loss: 8.9457e-05
Epoch 8/50
1604/1604 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 1.0278e-04 - val_loss: 8.2555e-05
Epoch 9/50
1604/1604 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 7.9304e-05 - val_loss: 8.5756e-05
Epoch 10/50
1604/1604 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 7.2813e-05 - val_loss: 9.0452e-05
Epoch 11/50
1604/1604 ━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
# Trích xuất đặc trưng ẩn (latent features)
encoder = models.Sequential(autoencoder.layers[:3])  # ví dụ lấy 3 lớp đầu
latent_features = encoder.predict(X_scaled)


3563/3563 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step


In [ ]:
# Hàm đề xuất bài hát dựa trên mô hình Autoencoder
def recommend_songs_dl(song_name, n=5):
    song_idx = df[df['track_name'].str.lower() == song_name.lower()].index
    if len(song_idx) == 0:
        return "Không tìm thấy bài hát!"
    
    song_vec = latent_features[song_idx[0]].reshape(1, -1)
    sims = cosine_similarity(song_vec, latent_features).flatten()
    
    df['similarity'] = sims
    recs = df.sort_values('similarity', ascending=False).iloc[1:n+1]
    return recs[['track_name', 'artists', 'track_genre', 'similarity']]


In [ ]:
# Ví dụ 1 sử dụng hàm đề xuất
recommendations = recommend_songs_dl('Comedy', n=5)
print(recommendations)


                                track_name      artists        track_genre  \
102150                              Comedy  Gen Hoshino         songwriter   
99151                               Comedy  Gen Hoshino  singer-songwriter   
62102                               Comedy  Gen Hoshino              j-pop   
357     Look For The Good (Single Version)   Jason Mraz           acoustic   
850                              Pop Virus  Gen Hoshino           acoustic   

        similarity  
102150    1.000000  
99151     1.000000  
62102     1.000000  
357       0.995269  
850       0.994691  


In [ ]:
# Ví dụ 2 sử dụng hàm đề xuất
recommendations = recommend_songs_dl('To Begin Again', n=5)
print(recommendations)


             track_name         artists track_genre  similarity
45035  Linda Mi Cholita    William Luna      guitar    0.998025
72486         Transcend  Fit For A King   metalcore    0.997224
40154   Tempo de Vencer          Jamily      gospel    0.997040
9353    Tempo de Vencer          Jamily      brazil    0.997040
1377             Llueve   Jorge Drexler    afrobeat    0.997039


In [ ]:
# Ví dụ 3 sử dụng hàm đề xuất
recommendations = recommend_songs_dl('Hold On', n=5)
print(recommendations)

              track_name           artists track_genre  similarity
954              Hold On  Chord Overstreet    acoustic    0.999980
70452                 分裂          Jay Chou    mandopop    0.998896
9290   Incomparável Amor     Gabriel Brito      brazil    0.998740
40146  Incomparável Amor     Gabriel Brito      gospel    0.998740
38544         Tid tröste        Mando Diao      garage    0.998702
